In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os
os.chdir('..')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 20

from scipy.stats import expon, gamma

import clds

# SEIR model with Gamma distributed recovery

In [ ]:
np.random.seed(seed=0)
dt = 0.01
# Sample gamma at dt substeps per day
def make_gamma(a, b=1, dt=dt):
    p_g = gamma(a=a, scale=b)
    n_days = a*b*5 # limit support
    t_range = np.linspace(0, n_days ,num=(n_days/dt)+1)
    y_g = p_g.pdf(t_range)
    return y_g/y_g.sum(), t_range
    


## Recovery distribution I -> R: ground-truth and Exponential fit

In [ ]:
# Sample distributions at discrete timesteps for simulation
# crop to a maximum of n_days between infection and recovery
b_ir = 0.99
a_ir = 2/b_ir # Gamma location parameter
yi2r, t_range = make_gamma(a=a_ir, b=b_ir)

plt.plot(t_range, yi2r, 'b-', alpha=0.6, label='gamma')
plt.xlabel('days until recovery')
plt.ylabel('density')
plt.legend()
plt.grid('on')
plt.title('Recovery PDF');

## Incubation time distribution E -> I: ground-truth and exponential fit

In [ ]:
b_ei = 0.1 # Gamma scale parameter (lower -> shaper peak)
a_ei = 12/b_ei # Gamma location parameter (higher -> higher delay)
ye2i, t_range = make_gamma(a_ei, b=b_ei)

plt.plot(t_range, ye2i, 'b-', alpha=0.6, label='gamma')
plt.xlabel('incubation period')
plt.ylabel('density')
plt.legend()
plt.grid('on')
plt.title('Incubation PDF');

## Simulation Setup

In [ ]:
# simulation
n_days = 365*2
dt=0.05 # step length in days

# epidemic parameters
N=1e7
R0 = 2.78
ei_b = 0.1
ei_a = 12./ei_b
ir_b = 0.99
ir_a = 2./ir_b

# switching
q = 0.175 # lockdown effectiveness: beta_ = q * beta+
steps_high = 2
steps_low = 14-steps_high
lockdown = 30 # number of days of unmitigated spread before prolonged lockdown
switching = int(lockdown + 20) # first day of periodic switching

# outer supervisory control
alpha_x = 0.4 # hystheresis for increasing duty cycle
alpha_y = 0. # hystheresis for decreasing duty cycle
x_init = 0
x_max = steps_high + steps_low
period = x_max

ye2i, t_range_e2i = make_gamma(a=ei_a, b=ei_b, dt=dt) # distribution of E->I delays
yi2r, t_range_i2r = make_gamma(a=ir_a, b=ir_b, dt=dt) # distribution of I-> delays
mean_ei_gamma = np.sum(t_range_e2i * ye2i)
mean_ir_gamma = np.sum(t_range_i2r * yi2r)
serial_interval = int(mean_ei_gamma + mean_ir_gamma)

# build simulation: run open-loop and closed-loop simulations simultaneously
# order='sequential' as opposed to concurrent updates the scratchpad in each step between subsequently added agents; in this case the agent order matters!
env = clds.Composite(order='sequential') 

# periodic switching controlled by outer loop
fpsp_cl = clds.agents.BatchFPSP(beta_high=R0, 
                        beta_low= R0*q,
                        steps_high='x',
                        steps_low='y',
                        suppression_start=lockdown,
                        switching_start=switching)
env.add(fpsp_cl, out='fpsp_cl', pre= lambda x: {'x': x['outer'][:,0], 'y': x['outer'][:,1]})

# open-loop periodic switching with fixed duty cycle
fpsp_ol = clds.agents.BatchFPSP(beta_high=R0, 
                        beta_low= R0*q, 
                        steps_high=steps_high, 
                        steps_low=steps_low,
                        suppression_start=lockdown,
                        switching_start=switching)
env.add(fpsp_ol, out='fpsp_ol')

# dynamics under open-loop control
model_ol =  clds.agents.SerialSEIR(ye2i, 
                    yi2r, 
                    N=N,
                    i0=500/6, 
                    e0=0, 
                    R0='fpsp_ol', 
                    dt=dt)
env.add(model_ol, out='model_ol')

# dynamics under closed-loop control
model_cl =  clds.agents.SerialSEIR(ye2i, 
                    yi2r, 
                    N=N,
                    i0=500/6, 
                    e0=0, 
                    R0='fpsp_cl', 
                    dt=dt)
env.add(model_cl, out='model_cl')


# outer loop input: I
u = clds.Lambda(reset_fn= lambda: 0, step_fn= lambda x: x['model_cl'][2]) # batch, channel
env.add(u, out='o')

# outer supervisory control
outer_loop = clds.agents.BatchOuterLoopFPSP(start=switching,
                            o='o', 
                            period=period, 
                            x_init=x_init,
                            x_max=x_max, 
                            alpha_x=alpha_x, 
                            alpha_y=alpha_y)
env.add(outer_loop, out='outer')

## Simulation Execution

In [ ]:
# execute simualtion
o = [env.reset()] + [env.step()[0] for _ in range(n_days)]
X_ol = np.array([x['model_ol'] for x in o])
X_cl = np.array([x['model_cl'] for x in o])

## Plot Transition Distribution

In [ ]:
def sequential_prob(a, b):
    q = np.zeros(a.shape[0] + b.shape[0])
    ab = np.meshgrid(a, b)
    ab = ab[0] * ab[1]
    for y in range(ab.shape[0]):
        for x in range(ab.shape[1]):
            q[x+y] += ab[y, x]
    return q

fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(t_range_e2i, ye2i, label='incubation (gamma, mean={:.0f})'.format(mean_ei_gamma))
ax.plot(t_range_i2r, yi2r, label='recovery (gamma, mean={:.0f})'.format(mean_ir_gamma))
e2r = sequential_prob(ye2i, yi2r)
ax.plot(np.arange(e2r.shape[0])*dt, e2r, '--', alpha=0.75, label='infection to recovery'.format(mean_ir_gamma))
ax.set_xlim([0, 28])
ax.set_xlabel('days')
ax.set_ylabel('density')
ax.legend()
ax.grid('on')
ax.set_title('Incubation and recovery PDFs');
plt.savefig(dpi=300, fname=f"results/agent_seir_E_{mean_ei_gamma:.0f}_{ei_b:.2f}_I_{mean_ir_gamma:.0f}_{ir_b:.2f}_pdf.png")
plt.savefig(dpi=300, fname=f"results/agent_seir_E_{mean_ei_gamma:.0f}_{ei_b:.2f}_I_{mean_ir_gamma:.0f}_{ir_b:.2f}_pdf.eps")

## Plot Open Loop Simulation

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(X_ol[:-1,1]/N*100, label='E')
ax.plot(X_ol[:-1,2]/N*100, label='I')
ylim = ax.get_ylim()
ax.plot([lockdown, lockdown], [ylim[0], ylim[1]], '-b', alpha=0.5, label=f'lockdown T={int(lockdown)}')
ax.plot([switching, switching], [ylim[0], ylim[1]], '-r', alpha=0.5, label=f'switching T={int(switching)}')
#ax.legend()
ax.grid('on')
ax.set_xlabel('days')
ax.set_ylabel('% of population')
ax.set_title(f"Open-loop FPSP-({steps_high},{steps_low})")
plt.savefig(dpi=300, fname=f"results/agent_seir_E_{mean_ei_gamma:.0f}_{ei_b:.2f}_I_{mean_ir_gamma:.0f}_{ir_b:.2f}_T_{period}_q_{q:.3f}_open_loop.png")
plt.savefig(dpi=300, fname=f"results/agent_seir_E_{mean_ei_gamma:.0f}_{ei_b:.2f}_I_{mean_ir_gamma:.0f}_{ir_b:.2f}_T_{period}_q_{q:.3f}_open_loop.eps")

## Plot Outer Supervisory Loop Simulation

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(X_cl[:-1,1]/N*100, label='E')
ax.plot(X_cl[:-1,2]/N*100, label='I')
ylim = ax.get_ylim()
ax.plot([lockdown, lockdown], [ylim[0], ylim[1]], '-b', alpha=0.5, label=f'lockdown T={int(lockdown)}')
ax.plot([switching, switching], [ylim[0], ylim[1]], '-r', alpha=0.5, label=f'switching T={int(switching)}')
ax.legend()
ax.grid('on')
ax.set_xlabel('days')
ax.set_ylabel('% of population')
ax.set_title(f"Slow outer supervisory control")
#plt.tight_layout()
plt.savefig(dpi=300, fname=f"results/agent_seir_E_{mean_ei_gamma:.0f}_{ei_b:.2f}_I_{mean_ir_gamma:.0f}_{ir_b:.2f}_T_{period}_q_{q:.3f}_closed_loop.png")
plt.savefig(dpi=300, fname=f"results/agent_seir_E_{mean_ei_gamma:.0f}_{ei_b:.2f}_I_{mean_ir_gamma:.0f}_{ir_b:.2f}_T_{period}_q_{q:.3f}_closed_loop.eps")

## Plot Combined Figure

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(30, 8))
labels = ['S', 'E', 'I', 'R']

ax = axes[0]
ax.plot(t_range_e2i, ye2i, label='incubation (gamma, mean={:.0f})'.format(mean_ei_gamma))
ax.plot(t_range_i2r, yi2r, label='recovery (gamma, mean={:.0f})'.format(mean_ir_gamma))
e2r = sequential_prob(ye2i, yi2r)
ax.plot(np.arange(e2r.shape[0])*dt, e2r, '--', alpha=0.75, label='infection to recovery'.format(mean_ir_gamma))
ax.set_xlim([0, 28])
ax.set_xlabel('days')
ax.set_ylabel('density')
ax.legend()
ax.grid('on')
ax.set_title('Incubation and recovery PDFs');

ax = axes[1]
ax.plot(X_ol[:-1,1]/N*100, label='E')
ax.plot(X_ol[:-1,2]/N*100, label='I')
ylim = ax.get_ylim()
ax.plot([lockdown, lockdown], [ylim[0], ylim[1]], '-b', alpha=0.5, label=f'lockdown T={int(lockdown)}')
ax.plot([switching, switching], [ylim[0], ylim[1]], '-r', alpha=0.5, label=f'switching T={int(switching)}')
#ax.legend()
ax.grid('on')
ax.set_xlabel('days')
ax.set_ylabel('% of population')
ax.set_title(f"Open-loop FPSP-({steps_high},{steps_low})")

ax = axes[2]
ax.plot(X_cl[:-1,1]/N*100, label='E')
ax.plot(X_cl[:-1,2]/N*100, label='I')
ylim = ax.get_ylim()
ax.plot([lockdown, lockdown], [ylim[0], ylim[1]], '-b', alpha=0.5, label=f'lockdown T={int(lockdown)}')
ax.plot([switching, switching], [ylim[0], ylim[1]], '-r', alpha=0.5, label=f'switching T={int(switching)}')
ax.legend()
ax.grid('on')
ax.set_xlabel('days')
ax.set_ylabel('% of population')
ax.set_title(f"Slow outer supervisory control")
#plt.tight_layout()
plt.savefig(dpi=300, fname=f"results/agent_seir_E_{mean_ei_gamma:.0f}_{ei_b:.2f}_I_{mean_ir_gamma:.0f}_{ir_b:.2f}_T_{period}_q_{q:.3f}_combined.png")
plt.savefig(dpi=300, fname=f"results/agent_seir_E_{mean_ei_gamma:.0f}_{ei_b:.2f}_I_{mean_ir_gamma:.0f}_{ir_b:.2f}_T_{period}_q_{q:.3f}_combined.eps")